In [1]:
# Problem 1

from gurobipy import *
import pandas as pd
import numpy as np
import time

start = time.time()
datafile = 'D:/lessons/OR/data/OR109-2_case02_data_s1.xlsx'

## import excel datas ##
Demand = pd.read_excel(datafile ,'Demand', index_col = 0)
IniInv = pd.read_excel(datafile,'Initial inventory', index_col = 0)
ShipCost = pd.read_excel(datafile,'Shipping cost', index_col = 0)
InTrans = pd.read_excel(datafile,'In-transit', index_col = 0)
CBMSize = pd.read_excel(datafile,'Size', index_col = 0)
BasicCost = pd.read_excel(datafile,'Price and cost', index_col = 0)
Shortage = pd.read_excel(datafile,'Shortage', index_col = 0)
LowerBound = pd.read_excel(datafile,'Bounds', index_col = 0)
Conflict = pd.read_excel(datafile,'Conflict', index_col = 0)
Vendorship = pd.read_excel(datafile,'Vendor-Product', index_col = 0)


Vendorship = Vendorship.T
Vendorship = np.array(Vendorship)
Vendorship = Vendorship.tolist()
Vendorship = Vendorship[0]

VendorCost = pd.read_excel(datafile,'Vendor cost', index_col = 0)
VendorCost = np.array(VendorCost)
VendorCost = VendorCost.tolist()
for i in range(len(VendorCost)):
    VendorCost[i] = VendorCost[i][0]

vendorNum = len(VendorCost)
Fender = []
for i in range(vendorNum):
    Fender.append([])

Demand = np.array(Demand)
Demand = Demand.tolist()

LowerBound = LowerBound.T
LowerBound = np.array(LowerBound)
LowerBound = LowerBound.tolist()
LowerBound = LowerBound[0]

itemNum = len(LowerBound)
time = len(Demand[0])

## extract important data
SalesPrice = BasicCost.iloc[:,0]
PurCost = BasicCost.iloc[:,1]
HoldingCost = BasicCost.iloc[:,2]
ExpressVCost = ShipCost.iloc[:,0]
AirVCost = ShipCost.iloc[:,1]
BackorderCost = Shortage.iloc[:,0]
LostSales = Shortage.iloc[:,1]
BackorderPer = Shortage.iloc[:,2]

Conflict = np.array(Conflict)
Conflict = Conflict.tolist()
print(Vendorship)
ConflictNum = len(Conflict)



for i in range(itemNum):
    indexx = Vendorship[i]
    Fender[indexx - 1].append(i)

productNum = []
for i in range(1, itemNum + 1):
    productNum.append(i)
    
monthNum = []
for i in range(time):
    monthNum.append(i)
    
methodNum = ["Express", "Air", "Ocean"]

# Data we can (or may) use:
#     SalesPrice,
#     PurCost,
#     HoldingCost
#     Demand,
#     IniInv,
#     FixedCost(E=0,A=1,O=2)(Imported below)
#     AirVCost,
#     ExpressVCost,
#     InTrans,
#     CBMSize,
#     BackorderCost,
#     LostSales,
#     BackorderPer

M = 1000000000



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [2]:
eg4 = Model("eg4")

stockLevel = []
for i in range(itemNum):
    stockLevel.append([])
for i in range(itemNum):
    stockLevel[i].append(IniInv.iloc[i,0])
for i in range(itemNum):
    for t in range(time):
        stockLevel[i].append(eg4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'SL'+ str(i+1)+str(t+1)))


## methods of shipping method Mi,j means 訂貨量 using method M for product i at order time j 
## expressQ/ airQ/ oceanQ stands for quantity from express/ air/ ocean method when it's delivered


## create expressQ
expressQ = []
for i in range(itemNum):
    expressQ.append([])
    
for i in range(itemNum):
    for j in range(time):
        expressQ[i].append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = 'EQ'+ str(i+1)+str(j+1)))
for i in range(itemNum):
    expressQ[i][0] = 0
    
## create airQ
airQ = []
for i in range(itemNum):
    airQ.append([])
    
for i in range(itemNum):
    for j in range(time):
        airQ[i].append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = 'AQ'+ str(i+1)+str(j+1)))
for i in range(itemNum):
    airQ[i][0] = 0
    airQ[i][1] = 0
            
## create oceanQ
oceanQ = []
for i in range(itemNum):
    oceanQ.append([])
    
for i in range(itemNum):
    for j in range(time):
        oceanQ[i].append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = 'OQ'+ str(i+1)+str(j+1)))
for i in range(itemNum):
    oceanQ[i][0] = 0
    oceanQ[i][1] = 0
    oceanQ[i][2] = 0

## create in-transit table intT
intT = []
for i in range(itemNum):
    intT.append([])
for i in range(itemNum):
    for j in range(time):
        intT[i].append(0)
    
for i in range(itemNum):
     for j in range(3):
        intT[i][j] =  InTrans.iloc[i,j]

## create Xi,t 
xit = []
for i in range(itemNum):
    xit.append([])
    
for i in range(itemNum):
    for j in range(time):
        xit[i].append(0)
        
## create Di,t 
D = []
for i in range(itemNum):
    D.append([])
    
for i in range(itemNum):
    for j in range(time):
        D[i].append(eg4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "D"+ str(i+1) + str(j+1)))

## create Lost[i],[t] 
Lost = []
for i in range(itemNum):
    Lost.append([])
    
for i in range(itemNum):
    for j in range(time):
        Lost[i].append(eg4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "Lost"+ str(i+1) + str(j+1)))


## set binary variable
zE = []
zA = []
zO = []
for i in range(time):
    zE.append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "zE"+ str(i+1)))
    zA.append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "zA"+ str(i+1)))
    zO.append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "zO"+ str(i+1)))

## Fixedcost  (EXPRESS , AIR , OCEAN)  
FixedCost = [100, 80, 50]

ss = []

for i in range(itemNum):
    ss.append([])

for i in range(itemNum):
    for t in range(time):
        ss[i].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "ss"+str(i+1)+ str(t+1)))

## calculate holding cost
## we took out the ssi after stocklevel
HoldingCostObj = quicksum(HoldingCost.iloc[i] * quicksum(stockLevel[i][t+1] for t in range(time)) for i in range(itemNum))
## calculate fixed cost
FixedCostObj = quicksum(FixedCost[0] * zE[i] + FixedCost[1] * zA[i] + FixedCost[2] * zO[i]  for i in range(time))

## calculate unfixed cost for method air and express 
## for air 
AVCObj = quicksum(AirVCost.iloc[i] * quicksum(airQ[i][t] for t in range(time)) for i in range(itemNum))
## for express
EVCObj = quicksum(ExpressVCost.iloc[i] * quicksum(expressQ[i][t] for t in range(time)) for i in range(itemNum))
UnfixedCostObj = AVCObj + EVCObj

## MerchCost
MerchCostObj = quicksum(quicksum(expressQ[i][t] + airQ[i][t] + oceanQ[i][t] for t in range(time))* PurCost.iloc[i] for i in range(itemNum))

## for container cost:
w = []
for t in range(time):
    w.append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = "w[" + str(t+1)+ "]"))

containerCost = 2750 * quicksum(w[t] for t in range(time))

LostSaleObj = quicksum(quicksum(Lost[i][t] for t in range(time)) * (1 - BackorderPer.iloc[i])* LostSales.iloc[i] for i in range(itemNum))
BackOrderObj = quicksum(quicksum(Lost[i][t] for t in range(time)) * BackorderPer.iloc[i] * BackorderCost.iloc[i] for i in range(itemNum))


l = []
for i in range(itemNum):
    l.append([])
    for t in range(time):
        l[i].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "l" + str(i+1) + str(t+1)))

for t in range(time):
    eg4.addConstrs(stockLevel[i][t+1] - Lost[i][t] == stockLevel[i][t] + intT[i][t] + expressQ[i][t] + airQ[i][t] + oceanQ[i][t] - D[i][t]for i in range(itemNum))
    eg4.addConstr(quicksum(expressQ[i][t] for i in range(itemNum)) <= M * zE[t])
    eg4.addConstr(quicksum(airQ[i][t] for i in range(itemNum))<= M * zA[t])
    eg4.addConstr(quicksum(oceanQ[i][t] for i in range(itemNum))<= M * zO[t])
    eg4.addConstr(w[t] >= quicksum(oceanQ[i][t] * CBMSize.iloc[i] for i in range(itemNum)) / 30)
    eg4.addConstrs(Lost[i][t] <= M *(ss[i][t]) for i in range(itemNum))
    eg4.addConstrs(stockLevel[i][t+1] <= M *(1 - ss[i][t]) for i in range(itemNum))
    
    if t == time - 1:
        pass
    elif t == time - 2 :
        eg4.addConstrs(expressQ[i][t+1] >= LowerBound[i] * l[i][t] for i in range(itemNum))
        eg4.addConstrs(expressQ[i][t+1] <= M * l[i][t] for i in range(itemNum))
    elif t == time - 3:
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] >= LowerBound[i] * l[i][t] for i in range(itemNum))
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] <= M * l[i][t] for i in range(itemNum))
    else:
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] + oceanQ[i][t+3] >= LowerBound[i] * l[i][t] for i in range(itemNum))
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] + oceanQ[i][t+3] <= M * l[i][t] for i in range(itemNum))

        

# vendor binary
vendorB = []
for j in range(vendorNum):
    vendorB.append([])
    for k in range(time):
        vendorB[j].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "vendor"+ str(j+1) + str(k+1)))

for t in range(time):
    if t == time-1:
        pass
    elif t == time-2:
        eg4.addConstrs(quicksum(expressQ[i][t+1] for i in Fender[j]) <= M * vendorB[j][t] for j in range(vendorNum))
    elif t == time-3:
        eg4.addConstrs(quicksum(expressQ[i][t+1] + airQ[i][t+2] for i in Fender[j]) <= M * vendorB[j][t] for j in range(vendorNum))
    else:
        eg4.addConstrs(quicksum(expressQ[i][t+1] + airQ[i][t+2] + oceanQ[i][t+3] for i in Fender[j]) <= M * vendorB[j][t] for j in range(vendorNum))
        
VendorObj = quicksum(quicksum(vendorB[k][t] for t in range(time)) * VendorCost[k] for k in range(vendorNum))

  
    
ConflictAlpha = []
for k in range(ConflictNum):
    ConflictAlpha.append([])
    
for k in range(ConflictNum):
    for t in range(time):
        ConflictAlpha[k].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "alpha" + str(k+1) + str(t+1)))


# print("ConflictNum" , ConflictNum)
# print(time)
# print(len(ConflictAlpha))
for k in range(ConflictNum):
    for t in range(time):
        i = Conflict[k][0]-1
        j = Conflict[k][1]-1
#         print(k,t)
        if t == time - 1:
            pass
        elif t == time - 2 :
            eg4.addConstr(expressQ[i][t+1] <= M * ConflictAlpha[k][t])
            eg4.addConstr(expressQ[j][t+1] <= M * (1-ConflictAlpha[k][t]))
        elif t == time - 3:
            eg4.addConstr(expressQ[i][t+1] + airQ[i][t+2] <= M * ConflictAlpha[k][t])
            eg4.addConstr(expressQ[j][t+1] + airQ[j][t+2] <= M * (1-ConflictAlpha[k][t]))
        else:
            eg4.addConstr(expressQ[i][t] + airQ[i][t-1] + oceanQ[i][t-2] <= M * ConflictAlpha[k][t])
            eg4.addConstr(expressQ[j][t] + airQ[j][t-1] + oceanQ[j][t-2] <= M * (1-ConflictAlpha[k][t]))
    



for t in range(time):
    if t == 0:
        eg4.addConstrs(D[i][0] >= Demand[i][0] for i in range(itemNum))
    else:
        eg4.addConstrs(D[i][t] >= Demand[i][t] + BackorderPer.iloc[i] * Lost[i][t-1]  for i in range(itemNum))
        
objValue = HoldingCostObj + FixedCostObj + UnfixedCostObj + containerCost + MerchCostObj + LostSaleObj + BackOrderObj + VendorObj
eg4.setObjective(objValue, GRB.MINIMIZE)

eg4.optimize()

Academic license - for non-commercial use only - expires 2021-05-29
Using license file C:\Users\JAYPOPEYE\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 16004 rows, 21190 columns and 69342 nonzeros
Model fingerprint: 0xc1e65a05
Variable types: 7800 continuous, 13390 integer (5564 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+09]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5962 rows and 5164 columns
Presolve time: 0.13s
Presolved: 10042 rows, 16026 columns, 53297 nonzeros
Variable types: 4985 continuous, 11041 integer (3821 binary)

Root relaxation: objective 1.281651e+09, 5448 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [7]:
# Problem 1

from gurobipy import *
import pandas as pd
import numpy as np

datafile = 'D:/lessons/OR/data/OR109-2_case02_data_s2.xlsx'

## import excel datas ##
Demand = pd.read_excel(datafile ,'Demand', index_col = 0)
IniInv = pd.read_excel(datafile,'Initial inventory', index_col = 0)
ShipCost = pd.read_excel(datafile,'Shipping cost', index_col = 0)
InTrans = pd.read_excel(datafile,'In-transit', index_col = 0)
CBMSize = pd.read_excel(datafile,'Size', index_col = 0)
BasicCost = pd.read_excel(datafile,'Price and cost', index_col = 0)
Shortage = pd.read_excel(datafile,'Shortage', index_col = 0)
LowerBound = pd.read_excel(datafile,'Bounds', index_col = 0)
Conflict = pd.read_excel(datafile,'Conflict', index_col = 0)
Vendorship = pd.read_excel(datafile,'Vendor-Product', index_col = 0)


Vendorship = Vendorship.T
Vendorship = np.array(Vendorship)
Vendorship = Vendorship.tolist()
Vendorship = Vendorship[0]

VendorCost = pd.read_excel(datafile,'Vendor cost', index_col = 0)
VendorCost = np.array(VendorCost)
VendorCost = VendorCost.tolist()
for i in range(len(VendorCost)):
    VendorCost[i] = VendorCost[i][0]

vendorNum = len(VendorCost)
Fender = []
for i in range(vendorNum):
    Fender.append([])

Demand = np.array(Demand)
Demand = Demand.tolist()

LowerBound = LowerBound.T
LowerBound = np.array(LowerBound)
LowerBound = LowerBound.tolist()
LowerBound = LowerBound[0]

itemNum = len(LowerBound)
time = len(Demand[0])

## extract important data
SalesPrice = BasicCost.iloc[:,0]
PurCost = BasicCost.iloc[:,1]
HoldingCost = BasicCost.iloc[:,2]
ExpressVCost = ShipCost.iloc[:,0]
AirVCost = ShipCost.iloc[:,1]
BackorderCost = Shortage.iloc[:,0]
LostSales = Shortage.iloc[:,1]
BackorderPer = Shortage.iloc[:,2]

Conflict = np.array(Conflict)
Conflict = Conflict.tolist()
print(Vendorship)
ConflictNum = len(Conflict)
print("   conflict  ")
print(Conflict)


for i in range(itemNum):
    indexx = Vendorship[i]
    Fender[indexx - 1].append(i)

productNum = []
for i in range(1, itemNum + 1):
    productNum.append(i)
    
monthNum = []
for i in range(time):
    monthNum.append(i)
    
methodNum = ["Express", "Air", "Ocean"]

# Data we can (or may) use:
#     SalesPrice,
#     PurCost,
#     HoldingCost
#     Demand,
#     IniInv,
#     FixedCost(E=0,A=1,O=2)(Imported below)
#     AirVCost,
#     ExpressVCost,
#     InTrans,
#     CBMSize,
#     BackorderCost,
#     LostSales,
#     BackorderPer

M = 1000000000



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
   conflict  
[[86, 91], [87, 98], [67, 73], [24, 85], [59, 86], [41, 97], [2, 19], [48, 55], [61, 76], [60, 85], [11, 52], [26, 34], [27, 67], [84, 95], [47, 73], [30, 33], [15, 61], [81, 100]]


In [ ]:
eg4 = Model("eg4")

stockLevel = []
for i in range(itemNum):
    stockLevel.append([])
for i in range(itemNum):
    stockLevel[i].append(IniInv.iloc[i,0])
for i in range(itemNum):
    for t in range(time):
        stockLevel[i].append(eg4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'SL'+ str(i+1)+str(t+1)))


## methods of shipping method Mi,j means 訂貨量 using method M for product i at order time j 
## expressQ/ airQ/ oceanQ stands for quantity from express/ air/ ocean method when it's delivered


## create expressQ
expressQ = []
for i in range(itemNum):
    expressQ.append([])
    
for i in range(itemNum):
    for j in range(time):
        expressQ[i].append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = 'EQ'+ str(i+1)+str(j+1)))
for i in range(itemNum):
    expressQ[i][0] = 0
    
## create airQ
airQ = []
for i in range(itemNum):
    airQ.append([])
    
for i in range(itemNum):
    for j in range(time):
        airQ[i].append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = 'AQ'+ str(i+1)+str(j+1)))
for i in range(itemNum):
    airQ[i][0] = 0
    airQ[i][1] = 0
            
## create oceanQ
oceanQ = []
for i in range(itemNum):
    oceanQ.append([])
    
for i in range(itemNum):
    for j in range(time):
        oceanQ[i].append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = 'OQ'+ str(i+1)+str(j+1)))
for i in range(itemNum):
    oceanQ[i][0] = 0
    oceanQ[i][1] = 0
    oceanQ[i][2] = 0

## create in-transit table intT
intT = []
for i in range(itemNum):
    intT.append([])
for i in range(itemNum):
    for j in range(time):
        intT[i].append(0)
    
for i in range(itemNum):
     for j in range(3):
        intT[i][j] =  InTrans.iloc[i,j]

## create Xi,t 
xit = []
for i in range(itemNum):
    xit.append([])
    
for i in range(itemNum):
    for j in range(time):
        xit[i].append(0)
        
## create Di,t 
D = []
for i in range(itemNum):
    D.append([])
    
for i in range(itemNum):
    for j in range(time):
        D[i].append(eg4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "D"+ str(i+1) + str(j+1)))

## create Lost[i],[t] 
Lost = []
for i in range(itemNum):
    Lost.append([])
    
for i in range(itemNum):
    for j in range(time):
        Lost[i].append(eg4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "Lost"+ str(i+1) + str(j+1)))


## set binary variable
zE = []
zA = []
zO = []
for i in range(time):
    zE.append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "zE"+ str(i+1)))
    zA.append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "zA"+ str(i+1)))
    zO.append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "zO"+ str(i+1)))

## Fixedcost  (EXPRESS , AIR , OCEAN)  
FixedCost = [100, 80, 50]

ss = []

for i in range(itemNum):
    ss.append([])

for i in range(itemNum):
    for t in range(time):
        ss[i].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "ss"+str(i+1)+ str(t+1)))

## calculate holding cost
## we took out the ssi after stocklevel
HoldingCostObj = quicksum(HoldingCost.iloc[i] * quicksum(stockLevel[i][t+1] for t in range(time)) for i in range(itemNum))
## calculate fixed cost
FixedCostObj = quicksum(FixedCost[0] * zE[i] + FixedCost[1] * zA[i] + FixedCost[2] * zO[i]  for i in range(time))

## calculate unfixed cost for method air and express 
## for air 
AVCObj = quicksum(AirVCost.iloc[i] * quicksum(airQ[i][t] for t in range(time)) for i in range(itemNum))
## for express
EVCObj = quicksum(ExpressVCost.iloc[i] * quicksum(expressQ[i][t] for t in range(time)) for i in range(itemNum))
UnfixedCostObj = AVCObj + EVCObj

## MerchCost
MerchCostObj = quicksum(quicksum(expressQ[i][t] + airQ[i][t] + oceanQ[i][t] for t in range(time))* PurCost.iloc[i] for i in range(itemNum))

## for container cost:
w = []
for t in range(time):
    w.append(eg4.addVar(lb = 0, vtype = GRB.INTEGER, name = "w[" + str(t+1)+ "]"))

containerCost = 2750 * quicksum(w[t] for t in range(time))

LostSaleObj = quicksum(quicksum(Lost[i][t] for t in range(time)) * (1 - BackorderPer.iloc[i])* LostSales.iloc[i] for i in range(itemNum))
BackOrderObj = quicksum(quicksum(Lost[i][t] for t in range(time)) * BackorderPer.iloc[i] * BackorderCost.iloc[i] for i in range(itemNum))


l = []
for i in range(itemNum):
    l.append([])
    for t in range(time):
        l[i].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "l" + str(i+1) + str(t+1)))

for t in range(time):
    eg4.addConstrs(stockLevel[i][t+1] - Lost[i][t] == stockLevel[i][t] + intT[i][t] + expressQ[i][t] + airQ[i][t] + oceanQ[i][t] - D[i][t]for i in range(itemNum))
    eg4.addConstr(quicksum(expressQ[i][t] for i in range(itemNum)) <= M * zE[t])
    eg4.addConstr(quicksum(airQ[i][t] for i in range(itemNum))<= M * zA[t])
    eg4.addConstr(quicksum(oceanQ[i][t] for i in range(itemNum))<= M * zO[t])
    eg4.addConstr(w[t] >= quicksum(oceanQ[i][t] * CBMSize.iloc[i] for i in range(itemNum)) / 30)
    eg4.addConstrs(Lost[i][t] <= M *(ss[i][t]) for i in range(itemNum))
    eg4.addConstrs(stockLevel[i][t+1] <= M *(1 - ss[i][t]) for i in range(itemNum))
    
    if t == time - 1:
        pass
    elif t == time - 2 :
        eg4.addConstrs(expressQ[i][t+1] >= LowerBound[i] * l[i][t] for i in range(itemNum))
        eg4.addConstrs(expressQ[i][t+1] <= M * l[i][t] for i in range(itemNum))
    elif t == time - 3:
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] >= LowerBound[i] * l[i][t] for i in range(itemNum))
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] <= M * l[i][t] for i in range(itemNum))
    else:
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] + oceanQ[i][t+3] >= LowerBound[i] * l[i][t] for i in range(itemNum))
        eg4.addConstrs(expressQ[i][t+1] + airQ[i][t+2] + oceanQ[i][t+3] <= M * l[i][t] for i in range(itemNum))

        

# vendor binary
vendorB = []
for j in range(vendorNum):
    vendorB.append([])
    for k in range(time):
        vendorB[j].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "vendor"+ str(j+1) + str(k+1)))

for t in range(time):
    if t == time-1:
        pass
    elif t == time-2:
        eg4.addConstrs(quicksum(expressQ[i][t+1] for i in Fender[j]) <= M * vendorB[j][t] for j in range(vendorNum))
    elif t == time-3:
        eg4.addConstrs(quicksum(expressQ[i][t+1] + airQ[i][t+2] for i in Fender[j]) <= M * vendorB[j][t] for j in range(vendorNum))
    else:
        eg4.addConstrs(quicksum(expressQ[i][t+1] + airQ[i][t+2] + oceanQ[i][t+3] for i in Fender[j]) <= M * vendorB[j][t] for j in range(vendorNum))
        
VendorObj = quicksum(quicksum(vendorB[k][t] for t in range(time)) * VendorCost[k] for k in range(vendorNum))

  
    
ConflictAlpha = []
for k in range(ConflictNum):
    ConflictAlpha.append([])
    
for k in range(ConflictNum):
    for t in range(time):
        ConflictAlpha[k].append(eg4.addVar(lb = 0, vtype = GRB.BINARY, name = "alpha" + str(k+1) + str(t+1)))

        
        
        
        
        

# print("ConflictNum" , ConflictNum)
# print(time)
# print(len(ConflictAlpha))
for k in range(ConflictNum):
    for t in range(time):
        i = Conflict[k][0]-1
        j = Conflict[k][1]-1
#         print(k,t)
        if t == time - 1:
            pass
        elif t == time - 2 :
            eg4.addConstr(expressQ[i][t+1] <= M * ConflictAlpha[k][t])
            eg4.addConstr(expressQ[j][t+1] <= M * (1-ConflictAlpha[k][t]))
        elif t == time - 3:
            eg4.addConstr(expressQ[i][t+1] + airQ[i][t+2] <= M * ConflictAlpha[k][t])
            eg4.addConstr(expressQ[j][t+1] + airQ[j][t+2] <= M * (1-ConflictAlpha[k][t]))
        else:
            eg4.addConstr(expressQ[i][t] + airQ[i][t-1] + oceanQ[i][t-2] <= M * ConflictAlpha[k][t])
            eg4.addConstr(expressQ[j][t] + airQ[j][t-1] + oceanQ[j][t-2] <= M * (1-ConflictAlpha[k][t]))
    



for t in range(time):
    if t == 0:
        eg4.addConstrs(D[i][0] >= Demand[i][0] for i in range(itemNum))
    else:
        eg4.addConstrs(D[i][t] >= Demand[i][t] + BackorderPer.iloc[i] * Lost[i][t-1]  for i in range(itemNum))
        
objValue = HoldingCostObj + FixedCostObj + UnfixedCostObj + containerCost + MerchCostObj + LostSaleObj + BackOrderObj + VendorObj
eg4.setObjective(objValue, GRB.MINIMIZE)

eg4.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 16454 rows, 21424 columns and 70905 nonzeros
Model fingerprint: 0xf4c3f570
Variable types: 7800 continuous, 13624 integer (5798 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+09]
  Objective range  [5e+00, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5830 rows and 5119 columns
Presolve time: 0.13s
Presolved: 10624 rows, 16305 columns, 55311 nonzeros
Variable types: 4979 continuous, 11326 integer (4117 binary)

Root relaxation: objective 1.645042e+08, 5936 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6450e+08    0   37     

In [ ]:

# for i in range(itemNum):
#     xit[i][0] =  intT[i][0]
#     xit[i][1] =  expressQ[i][1].X + intT[i][1]
#     xit[i][2] =  expressQ[i][2].X + airQ[i][2].X + intT[i][2]
#     for t in range(3,time):
#         xit[i][t] = expressQ[i][t].X + airQ[i][t].X + oceanQ[i][t].X + intT[i][t]

# print("");
# endingInvDf = pd.DataFrame(endingInv)
# endingInvDf.columns = ["Ini"] + monthNum
# endingInvDf.index = productNum
# print("每個月份的Ending Inventory:")
# print(endingInvDf)
# print("")



# for i in range(itemNum):
#     for t in range(time):
#         ss[i][t] = ss[i][t].X
#     for t in range(1,time+1):
#         stockLevel[i][t] =  stockLevel[i][t].X
# print("----ssit----")
# for i in range(itemNum):
#     print(ss[i])
# print(" ")
# print("");
# StockLevelDf = pd.DataFrame(stockLevel)
# StockLevelDf.columns = ["Ini"] + monthNum
# StockLevelDf.index = productNum
# print("每個月份的Stock Level:")
# print(StockLevelDf)
# print("")
# print(" this is demand")
# for i in range(itemNum):
#     for t in range(time):
#         D[i][t] = D[i][t].X
# for i in range(itemNum):
#     print(" item "+str(i+1),end = " " )
#     print(D[i])
# xitDf = pd.DataFrame(xit)
# xitDf.columns = monthNum
# xitDf.index = productNum
# print("每個月份將會運來的所有存貨量:")
# print(xitDf)
# print("")

# for i in range(itemNum):
#     xit[i][0] =  intT[i][0]
#     xit[i][1] =  expressQ[i][1].X + intT[i][1]
#     xit[i][2] =  expressQ[i][2].X + airQ[i][2].X + intT[i][2]
#     for t in range(3,time):
#         xit[i][t] = expressQ[i][t].X + airQ[i][t].X + oceanQ[i][t].X + intT[i][t]
# print("----- w ---")
# for t in range(time):
#     w[t] = w[t].X
# print(w)
# print(" ")


# for i in range(itemNum):
#     for t in range(1,time):
#         expressQ[i][t] = expressQ[i][t].X
# expressQDf = pd.DataFrame(expressQ)
# expressQDf.columns = monthNum
# expressQDf.index = productNum
# print("會透過express的方式寄來的存貨:")
# print(expressQDf) 
# print("")

# for i in range(itemNum):
#     for t in range(2,time):
#         airQ[i][t] = airQ[i][t].X
# airQDf = pd.DataFrame(airQ)
# airQDf.columns = monthNum
# airQDf.index = productNum
# print("會透過air的方式寄來的存貨:")
# print(airQDf)
# print("")

# for i in range(itemNum):
#     for t in range(3,time):
#         oceanQ[i][t] = oceanQ[i][t].X
# oceanQDf = pd.DataFrame(oceanQ)
# oceanQDf.columns = monthNum
# oceanQDf.index = productNum
# print("會透過ocean的方式寄來的存貨:")
# print(oceanQDf)
# print("")

# OrderDecision = []
# for t in range(time-1):
#     OrderDecision.append([])
# for t in range(time-1):
#     for i in range(itemNum):
#         OrderDecision[t].append([])
# for t in range(time-1):
#     for i in range(itemNum):
#         if t + 1 < time: OrderDecision[t][i].append(expressQ[i][t+1])
#         if t + 2 < time: OrderDecision[t][i].append(airQ[i][t+2])
#         if t + 3 < time: OrderDecision[t][i].append(oceanQ[i][t+3])

# print("")            
# for i in range(itemNum):
#     for t in range(time):
#         Lost[i][t] = Lost[i][t].X
# LostDf = pd.DataFrame(Lost)
# LostDf.columns = monthNum
# LostDf.index = productNum
# print("Lost sales :")
# print(LostDf)
# print("")

# OrderDecisionDf = []
# for t in range(time-1):
#     OrderDecisionDf.append(pd.DataFrame(OrderDecision[t]))
#     OrderDecisionDf[t] = pd.DataFrame.transpose(OrderDecisionDf[t])
#     OrderDecisionDf[t].columns = productNum

# OrderDecisionDf[0].index = ["Express", "Air", "Ocean"]
# OrderDecisionDf[1].index = ["Express", "Air", "Ocean"]
# OrderDecisionDf[2].index = ["Express", "Air", "Ocean"]
# OrderDecisionDf[3].index = ["Express", "Air"]
# OrderDecisionDf[4].index = ["Express"]

# for t in range(5):
#     print("第" + str(t+3) + "個月須下的決策:")
#     print(OrderDecisionDf[t])
#     print("")